In [ ]:
import json
from llama_cpp import Llama
import numpy as np
import pandas as pd
import openai
import os

import logging

#del os.environ["GLOG_minloglevel"]
#logging.getLogger("/Users/hanlianlyu/llama2/TheBloke/Llama-2-7B-GGUF/llama-2-7b.Q6_K.gguf").setLevel(logging.CRITICAL)

llm = Llama(model_path="/Users/hanlianlyu/llama2/TheBloke/OpenOrca-Platypus2-13B-GGUF/openorca-platypus2-13B-Q4_K_M.gguf")

df = pd.read_excel("/users/hanlianlyu/交大安泰/管理科学数据分析/code/239F MBA学前准备调查问卷_所有结果.xlsx", sheet_name="Table1")

In [100]:
# Remove numbering and extra spaces
cleaned_headers = [header.split('、')[-1].strip() for header in df.columns]

# Manual simplification of some of the column namee
header_mapping = {
    "提交答卷时间": "提交时间",
    "学号（可在“交我办app”的“设置”中查看）": "学号",
    "请选出所有你学习过的数学相关内容": "学习过的数学内容",
    "请问你的工作岗位主要是什么": "工作岗位",
    "请问你工作的企业是否已经建立数据中台": "是否建立数据中台",
    "请问你是否愿意提供企业内脱敏后的数据给课程项目": "是否提供脱敏数据",
    "请问你是否了解和使用过ChatGPT": "了解ChatGPT",
    "请问你是否订阅或购买过ChatGPT-Plus / OpenAI-API / Replicate等服务": "订阅OpenAI服务",
    "请问你是否使用过类似的AI大语言模型": "使用AI大语言模型",
    "请提供至少一项，你对应用数据分析最感兴趣的具体议题（提示：请基于你的工作范围，行业公司，创业方向，研究兴趣等考虑并展开描述，不超过200字，必填）": "数据分析兴趣议题",
    "请问你期望从这门课学到什么内容（选填）": "期望学到的内容"
}

# Apply the mapping to the cleaned headers
simplified_headers = [header_mapping.get(header, header) for header in cleaned_headers]

# Update the dataframe with the new headers
df.columns = simplified_headers
pd.set_option('display.max_columns', None)
df.head(0)

,序号,提交时间,所用时间,来源,来源详情,来自IP,姓名,学号,学习过的数学内容,请选择符合你编程经验的选项,请选出所有你使用过的编程语言,请选出所有你使用过的Python数据分析相关的库和工具,请选出所有你使用过的在线编程工具,请问你在什么行业工作,工作岗位,是否建立数据中台,请问你是否了解数据治理,是否提供脱敏数据,了解ChatGPT,订阅OpenAI服务,使用AI大语言模型,数据分析兴趣议题,期望学到的内容,Trans_result,summarized,Trans_task


In [122]:
prompt = "Translate the Chinese passage into English: "
df["Trans_task"] = prompt + df["数据分析兴趣议题"].astype(str)

df_test = df["Trans_task"].iloc[0:5].to_frame()
df_test

,Trans_task
0,Translate the Chinese passage into English: 市场分析
1,Translate the Chinese passage into English: 如果在环境受限制，具体为有合规隐私等限制的情形下在有限的客户接触点中收集数据，帮助分析和画像建立
2,Translate the Chinese passage into English: 希望数据分析能够帮助我合理梳理日常工作在系统中产生的数据，做出相关报表帮助业务分析和提效
3,Translate the Chinese passage into English: 人工智能的应用与安全
4,Translate the Chinese passage into English: 消费者洞察


In [127]:
#prefix = "[INST]"
#suffix = "[/INST]"
prompt = f"""[INST] <<SYS>>
You are an AI translator, your answers should follow the requirement.
<</SYS>>
{prompt}[/INST]"""
def get_completion(prompt, model= llm , temperature=0.75):
    #formatted_prompt = f"{prefix}{prompt}{suffix}"
    formatted_prompt = prompt
    #messages = [{"role": "user", "content": formatted_prompt}]
    messages = f"{formatted_prompt} "
    #print(f'\nYour prompt: {prompt}\n')
    response = llm.create_completion(
        prompt=messages,
        temperature=temperature,
        top_p = 0.95,
        max_tokens= 1000
    )
    #return response.choices[0].message["content"]
    return response

response = df_test['Trans_task'].apply(lambda x:get_completion(x, temperature = 0) )
#response = df_test['Trans_task'].apply(lambda x:llm.create_completion(x, suffix=None, max_tokens=1000, temperature=0.7, top_p=0.95, logprobs=None, echo=False, stop=[], frequency_penalty=0.0, presence_penalty=0.0, repeat_penalty=1.1, top_k=40, stream=False, tfs_z=1.0, mirostat_mode=0, mirostat_tau=5.0, mirostat_eta=0.1, model=None, stopping_criteria=None, logits_processor=None, grammar=None) )
print('Creation Start\n')
#response = df_test['Trans_result'].apply(lambda x:llm(x, max_tokens = -1, echo=False, stop = ["\n"]))
print('Creation DONE')

Llama.generate: prefix-match hit

llama_print_timings:        load time = 14084.85 ms
llama_print_timings:      sample time =    33.92 ms /    43 runs   (    0.79 ms per token,  1267.61 tokens per second)
llama_print_timings: prompt eval time =   789.26 ms /     7 tokens (  112.75 ms per token,     8.87 tokens per second)
llama_print_timings:        eval time =  4298.11 ms /    42 runs   (  102.34 ms per token,     9.77 tokens per second)
llama_print_timings:       total time =  5183.68 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time = 14084.85 ms
llama_print_timings:      sample time =    55.36 ms /    70 runs   (    0.79 ms per token,  1264.47 tokens per second)
llama_print_timings: prompt eval time =  9690.56 ms /    69 tokens (  140.44 ms per token,     7.12 tokens per second)
llama_print_timings:        eval time =  7046.22 ms /    69 runs   (  102.12 ms per token,     9.79 tokens per second)
llama_print_timings:       total time = 16892.67 ms
Llama.gene

Creation Start

Creation DONE



llama_print_timings:        load time = 14084.85 ms
llama_print_timings:      sample time =    19.05 ms /    24 runs   (    0.79 ms per token,  1259.51 tokens per second)
llama_print_timings: prompt eval time =  1031.94 ms /    10 tokens (  103.19 ms per token,     9.69 tokens per second)
llama_print_timings:        eval time =  2412.91 ms /    23 runs   (  104.91 ms per token,     9.53 tokens per second)
llama_print_timings:       total time =  3497.53 ms


In [128]:

# Set the option to display the maximum column width
pd.set_option('display.max_colwidth', None)
# Set the print option to show everything
np.set_printoptions(threshold=np.inf)
#print(response)
response

0                                                                                                                                                                                                                                                           {'id': 'cmpl-33ea12a7-93ef-4861-9733-bfbf5107eddb', 'object': 'text_completion', 'created': 1695187643, 'model': '/Users/hanlianlyu/llama2/TheBloke/OpenOrca-Platypus2-13B-GGUF/openorca-platypus2-13B-Q4_K_M.gguf', 'choices': [{'text': '（Market Analysis）

The given Chinese text "市场分析" (Shìchǎng Fēnjī) can be translated into English as "Market Analysis."', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 17, 'completion_tokens': 42, 'total_tokens': 59}}
1    {'id': 'cmpl-5f16e8ba-036c-4abe-8bb9-ed8a5ea71543', 'object': 'text_completion', 'created': 1695187648, 'model': '/Users/hanlianlyu/llama2/TheBloke/OpenOrca-Platypus2-13B-GGUF/openorca-platypus2-13B-Q4_K_M.gguf', 'choices': [{'text': '。

If data is coll

In [129]:
df_test['summarized'] = response.apply(lambda x : x['choices'][0]['text'])

In [130]:
df_test

,Trans_task,summarized
0,Translate the Chinese passage into English: 市场分析,"（Market Analysis）\n\nThe given Chinese text ""市场分析"" (Shìchǎng Fēnjī) can be translated into English as ""Market Analysis."""
1,Translate the Chinese passage into English: 如果在环境受限制，具体为有合规隐私等限制的情形下在有限的客户接触点中收集数据，帮助分析和画像建立,"。\n\nIf data is collected in limited customer touchpoints under the circumstances of having compliant privacy restrictions, it helps with analysis and image building.\n\nIn a restricted environment with specific limitations, such as having compliant privacy regulations, collecting data in limited customer contact points can help with analysis and image construction."
2,Translate the Chinese passage into English: 希望数据分析能够帮助我合理梳理日常工作在系统中产生的数据，做出相关报表帮助业务分析和提效,"。\n\nI hope data analysis can help me reasonably organize the data generated in daily work within the system, and produce relevant reports to assist business analysis and improve efficiency."
3,Translate the Chinese passage into English: 人工智能的应用与安全,"（Artificial Intelligence Applications and Security）\n\nThe provided text is already in English.\n\nHowever, if you meant to translate it from Chinese, please provide the Chinese text."
4,Translate the Chinese passage into English: 消费者洞察,"感性体验\n\nTranslation: Consumer Insight, Sensory Experience"
